# IS424: Data Mining & Biz Analytics
### Team: G3T3
### Project: Predicting Loan Default based on Customer Profile
### Model selection: <font color='#0041C2'>Neural Network</font>
---

# 1. Setting up the notebook

In [26]:
import pandas as pd
import numpy as np

from imblearn.over_sampling import SMOTE
from category_encoders import TargetEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import recall_score, fbeta_score, roc_auc_score, make_scorer
from sklearn.model_selection import cross_val_score, StratifiedKFold

import tensorflow as tf
tf.autograph.set_verbosity(0)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import Recall, AUC, Precision
import tensorflow_addons as tfa

tf.get_logger().setLevel("INFO")

In [27]:
df_train = pd.read_csv("../dataset/train.csv")

y_train = df_train[["risk_flag"]]
x_train = df_train.drop("risk_flag", axis=1)

In [28]:
def target_encoding(df_x, df_y):
    x = df_x.copy()
    
    # Target Encoding — categorical columns with high cardinality: profession, city, state
    profession_target_enc = TargetEncoder()
    x["profession_encoded"] = profession_target_enc.fit_transform(x["profession"], df_y)
    
    city_target_enc = TargetEncoder()
    x["city_encoded"] = city_target_enc.fit_transform(x["city"], df_y)
    
    state_target_enc = TargetEncoder()
    x["state_encoded"] = state_target_enc.fit_transform(x["state"], df_y)
    
    x.drop("profession", axis=1, inplace=True)
    x.drop("city", axis=1, inplace=True)
    x.drop("state", axis=1, inplace=True)
    return x

scale_features = ['income','age','experience']
x_train = target_encoding(x_train, y_train)
scaler = MinMaxScaler()
x_train[scale_features] = scaler.fit_transform(x_train[scale_features])

/Users/jonw/miniforge3/envs/ml_env/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


# 2. Running base model

In [29]:
@tf.autograph.experimental.do_not_convert
def run_NN_model():
    fold_num = 1
    skf = StratifiedKFold(shuffle=True, n_splits=3, random_state=2021)
    scores = []

    for train, val in skf.split(x_train, y_train):
        input_shape = (9,)

        model = Sequential()
        model.add(Dense(120, input_shape=input_shape, activation='relu'))
        model.add(Dense(120, activation='relu'))
        model.add(Dense(120, activation='relu'))
        model.add(Dense(2, activation='softmax'))

        model.compile(loss='binary_crossentropy', 
                      optimizer='adam', 
                      metrics=["accuracy", Recall(), tfa.metrics.FBetaScore(num_classes=2, beta=2.0), AUC()])

        x_t, y_t = x_train.iloc[train], y_train.iloc[train]
        x_val, y_val = x_train.iloc[val], y_train.iloc[val]

        y_t = to_categorical(y_t, 2)
        y_val = to_categorical(y_val, 2)

        model_fit = model.fit(x_t, y_t, epochs=5, batch_size=64, verbose=0)
        result = model.evaluate(x_val, y_val, verbose=1)
        scores.append(result)

        fold_num += 1
        
run_NN_model()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
2100/2100 [==============================] - 1s 409us/step - loss: 0.3567 - accuracy: 0.8773 - recall_9: 0.8773 - fbeta_score: 0.4925 - auc_9: 0.9092
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and

In [30]:
loss = [ val[0] for val in scores ]
recall = [ val[2] for val in scores ]
fbeta_2 = [ val[3][0] for val in scores ]
auc = [ val[4] for val in scores ]
recall.append( sum(recall) / len(recall) )
fbeta_2.append( sum(fbeta_2) / len(fbeta_2) )
auc.append( sum(auc) / len(auc) )

score_df = pd.DataFrame(data=[recall, fbeta_2, auc], columns=['Fold 1','Fold 2','Fold 3', 'Average'],
                        index=['Recall','Fbeta2','AUC'])
display(score_df)

,Fold 1,Fold 2,Fold 3,Average
Recall,0.876815,0.876979,0.876949,0.876915
Fbeta2,0.972474,0.972700,0.972311,0.972495
AUC,0.907424,0.909464,0.909590,0.908826
